Columns and meanings:

timestamp: date and time (YYYY-MM-DD HH:MM:SS), every 10 seconds.

campaign: Campaign number groups several batches (e.g., 5–15) into one manufacturing cycle; the batches belonging to the same campaign were manufactured one after the other.

batch: Batch number identifies the batch of the final product.

code: Product code number defines the product sub-family to which the batch belongs. Every time series dataset file has the same product code and contains all batches within the same product code.

tbl_speed: Tablet press speed, it indicates when the process is running and when it has stopped, if there were many changes to this parameter or many stoppages, the material handling is challenging, which may indicate suboptimal product quality.

fom: Filling device speed in rotations per minute: similar to tablet press speed. If the process is running, so is the filling device. This parameter generally does not change and is only set during the start-up. If many changes (during the start-up) are observed, this again indicates potential difficulties with incoming material handling.

main_comp: Main compression force – mean value: the more constant this parameter is, the more homogeneous is the incoming material blend in terms of physical properties.

tbl_fill: Tablet fill depth: defines the volume of filled blended material to be compressed. If flow properties of material are poor, this parameter will vary throughout the batch and will consequently impact tablet hardness and weight.

SREL: Main compression force – standard relative deviation: this parameter is calculated by the tablet press itself by using main compression force mean values. It gives an indication of how uniform the tablets compacted are.

pre_comp: Pre-compression force – mean value: if pre-compression force is used for tablet compaction, this parameter will be greater than 1 and will give a similar indication as main compression force. It is not readily used for the product in the scope.

produced: Good production: all acceptable tablets that have been produced at that particular timestamp. ESTA ME PREOCUPA

waste: Bad production: tablets that do not pass the set tablet press parameters (i.e., max % deviation from the set main compression force – mean value). This is also a cumulative parameter and gives information about all rejected tablets at that particular time. ESTA ME PREOCUPA

cyl_main:Cylindrical height – main compression: cylindrical height of the tablet (main compression station) in mm. The height and hardness of the tablet are changed by changing the cylindrical height.

cyl_pre: Cylindrical height – pre-compression: cylindrical height of the tablet (pre-compression station) in mm.

stiffness: Bottom punch stiffness in Newton: when the limit is reached, the press is stopped with suitable diagnosis. An equipment parameter.

ejection: Maximum tablet ejection force: if this parameter rises, the tablet ejection friction is higher, which could mean that some minor sticking of the tablet has occurred on the tablet tooling.

start_time: time at which each batch has started production (this value is repeated across the batch)

timeSince: Time since the batch has started, this is the substraction between start_time and timestamp. The result is in days, hours, minutes ad seconds.

timeSinceSec: Same value as timeSince but the value has been converted to seconds.

impurities_total: This is the target. percentage of impurities, this lab result is taken at the end of the process. The value is between 0 and 1.

In [253]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import time
from datetime import datetime

In [254]:
# Combine all the csv files into one big dataframe.

path = "Process Time Series/"
allfiles = glob.glob(path + "*.csv")

li = []

for filename in allfiles:
    df = pd.read_csv(filename, index_col=None, header=0, sep=';')
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [255]:
frame

,timestamp,campaign,batch,code,tbl_speed,fom,main_comp,tbl_fill,SREL,pre_comp,produced,waste,cyl_main,cyl_pre,stiffness,ejection
0,2019-09-06 04:08:23,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0
1,2019-09-06 04:08:33,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0
2,2019-09-06 04:08:43,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0
3,2019-09-06 04:08:53,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0
4,2019-09-06 04:09:03,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4720203,2019-09-03 15:13:29,81,367,8,0.0,0.0,11.7,6.80,6.3,0.0,1078.0,19982.0,1.44,5.0,233.0,236.0
4720204,2019-09-03 15:13:39,81,367,8,0.0,0.0,11.7,6.80,6.3,0.0,1078.0,19982.0,1.44,5.0,233.0,236.0
4720205,2019-09-03 15:13:49,81,367,8,0.0,0.0,11.7,6.80,6.3,0.0,1078.0,19982.0,1.44,5.0,233.0,236.0
4720206,2019-09-03 15:13:59,81,367,8,0.0,0.0,11.7,6.80,6.3,0.0,1078.0,19982.0,1.44,5.0,233.0,236.0


In [256]:
#We have two diferent date formats in timestamp so I am converting them to datetime format.

frame['timestamp'] = pd.to_datetime(frame['timestamp'], 
                                    format='%Y%m%d %H:%M:%S', 
                                    errors="coerce").fillna(pd.to_datetime(frame['timestamp'], 
                                                                           format='%d%m%Y %H:%M', 
                                                                           errors="coerce"))

In [257]:
# Searching for empty values in timestamp

frame['timestamp'].isna().sum()

0

In [258]:
frame.shape

(4720208, 16)

In [259]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4720208 entries, 0 to 4720207
Data columns (total 16 columns):
 #   Column     Dtype         
---  ------     -----         
 0   timestamp  datetime64[ns]
 1   campaign   int64         
 2   batch      int64         
 3   code       int64         
 4   tbl_speed  float64       
 5   fom        float64       
 6   main_comp  float64       
 7   tbl_fill   float64       
 8   SREL       float64       
 9   pre_comp   float64       
 10  produced   float64       
 11  waste      float64       
 12  cyl_main   float64       
 13  cyl_pre    float64       
 14  stiffness  float64       
 15  ejection   float64       
dtypes: datetime64[ns](1), float64(12), int64(3)
memory usage: 576.2 MB


In [260]:
# Searching for missing values

columnsMissingValues = frame.columns[frame.isna().any().tolist()]
frame[columnsMissingValues].isna().sum()

tbl_speed     2206
fom          17291
main_comp     2205
tbl_fill      2191
SREL          2244
pre_comp      2207
produced      2494
waste         2197
cyl_main      2301
cyl_pre       2494
stiffness     2207
ejection      2254
dtype: int64

In [261]:
# We have 4.7 million rows, we should probably drop the missing values

frame = frame.dropna()
frame.shape

(4702581, 16)

In [262]:
# We dropped around 20,000 rows

In [263]:
# I have to add the target to each one of the batches

laboratory = pd.read_csv("Laboratory.csv", sep=";")

In [264]:
labResults = laboratory[['batch', 'impurities_total']]

In [265]:
labResults.head()

,batch,impurities_total
0,1,0.33
1,2,0.34
2,3,0.28
3,4,0.30
4,5,0.31


In [266]:
#I merge my process data to the target (impurities_total)

df = pd.merge(frame, labResults, on="batch")

In [267]:
df.head()

,timestamp,campaign,batch,code,tbl_speed,fom,main_comp,tbl_fill,SREL,pre_comp,produced,waste,cyl_main,cyl_pre,stiffness,ejection,impurities_total
0,2019-09-06 04:08:23,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,0.11
1,2019-09-06 04:08:33,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,0.11
2,2019-09-06 04:08:43,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,0.11
3,2019-09-06 04:08:53,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,0.11
4,2019-09-06 04:09:03,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,0.11


In [268]:
df[df['batch']==1004]['timestamp'].min()

Timestamp('2021-07-05 03:45:00')

In [269]:
#I get an array of all the batches, then I use the length of the array to iterate through them

batches = df['batch'].unique()
batches

array([368, 369,  82, ..., 353, 123, 367])

In [270]:
#I get a list of each batch and a list of the start time of each batch

batcheslist = []
timeslist = []
for i in range(len(batches)):
    batcheslist.append(i+1)
    timeslist.append(df[df['batch']==i+1]['timestamp'].min())

In [271]:
timeslist

[Timestamp('2018-11-13 06:16:33'),
 Timestamp('2018-11-13 09:43:07'),
 Timestamp('2018-11-13 13:05:30'),
 Timestamp('2018-11-13 16:06:07'),
 Timestamp('2018-11-13 19:02:59'),
 Timestamp('2018-11-13 22:13:16'),
 Timestamp('2018-11-14 01:12:17'),
 Timestamp('2018-11-14 04:15:38'),
 Timestamp('2018-11-14 07:32:19'),
 Timestamp('2018-11-14 10:53:28'),
 Timestamp('2018-11-14 14:14:39'),
 Timestamp('2018-11-14 17:27:08'),
 Timestamp('2018-11-14 20:31:13'),
 Timestamp('2018-11-14 23:51:01'),
 Timestamp('2018-11-15 03:11:40'),
 Timestamp('2018-11-18 22:34:33'),
 Timestamp('2018-11-19 17:10:55'),
 Timestamp('2018-11-20 11:40:26'),
 Timestamp('2018-11-18 22:28:52'),
 Timestamp('2018-11-19 07:32:41'),
 Timestamp('2018-11-19 16:58:07'),
 Timestamp('2018-11-22 23:59:14'),
 Timestamp('2018-11-23 07:13:56'),
 Timestamp('2018-11-23 14:14:27'),
 Timestamp('2018-11-25 22:40:54'),
 Timestamp('2018-11-22 23:07:27'),
 Timestamp('2018-11-23 02:37:59'),
 Timestamp('2018-11-23 08:51:53'),
 Timestamp('2018-11-

In [272]:
# I join the two lists into a dictionary and then create a dataframe

dict1 = {'batch':batcheslist, 'start_time':timeslist}
df_start = pd.DataFrame.from_dict(dict1)
df_start

,batch,start_time
0,1,2018-11-13 06:16:33
1,2,2018-11-13 09:43:07
2,3,2018-11-13 13:05:30
3,4,2018-11-13 16:06:07
4,5,2018-11-13 19:02:59
...,...,...
1000,1001,2021-06-05 00:20:00
1001,1002,2021-06-05 09:22:00
1002,1003,2021-05-06 18:36:00
1003,1004,2021-07-05 03:45:00


In [273]:
# I am doing a test to get the time since start time

df[df['batch']==1004]['timestamp'].min() - df_start[df_start['batch']==1]['start_time']

0   964 days 21:28:27
Name: start_time, dtype: timedelta64[ns]

In [274]:
# I merge tthe start time to the dataframe

df = pd.merge(df, df_start, on="batch")

In [275]:
df

,timestamp,campaign,batch,code,tbl_speed,fom,main_comp,tbl_fill,SREL,pre_comp,produced,waste,cyl_main,cyl_pre,stiffness,ejection,impurities_total,start_time
0,2019-09-06 04:08:23,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,0.11,2019-09-06 04:08:23
1,2019-09-06 04:08:33,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,0.11,2019-09-06 04:08:23
2,2019-09-06 04:08:43,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,0.11,2019-09-06 04:08:23
3,2019-09-06 04:08:53,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,0.11,2019-09-06 04:08:23
4,2019-09-06 04:09:03,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,0.11,2019-09-06 04:08:23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4702576,2019-09-03 15:13:29,81,367,8,0.0,0.0,11.7,6.80,6.3,0.0,1078.0,19982.0,1.44,5.0,233.0,236.0,0.08,2019-09-02 19:26:09
4702577,2019-09-03 15:13:39,81,367,8,0.0,0.0,11.7,6.80,6.3,0.0,1078.0,19982.0,1.44,5.0,233.0,236.0,0.08,2019-09-02 19:26:09
4702578,2019-09-03 15:13:49,81,367,8,0.0,0.0,11.7,6.80,6.3,0.0,1078.0,19982.0,1.44,5.0,233.0,236.0,0.08,2019-09-02 19:26:09
4702579,2019-09-03 15:13:59,81,367,8,0.0,0.0,11.7,6.80,6.3,0.0,1078.0,19982.0,1.44,5.0,233.0,236.0,0.08,2019-09-02 19:26:09


In [276]:
#I calculate the timeSince column, for each row, the result is in days, hours, minutes and seconds

df['timeSince'] = df['timestamp']-df['start_time']

In [277]:
df

,timestamp,campaign,batch,code,tbl_speed,fom,main_comp,tbl_fill,SREL,pre_comp,produced,waste,cyl_main,cyl_pre,stiffness,ejection,impurities_total,start_time,timeSince
0,2019-09-06 04:08:23,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,0.11,2019-09-06 04:08:23,0 days 00:00:00
1,2019-09-06 04:08:33,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,0.11,2019-09-06 04:08:23,0 days 00:00:10
2,2019-09-06 04:08:43,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,0.11,2019-09-06 04:08:23,0 days 00:00:20
3,2019-09-06 04:08:53,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,0.11,2019-09-06 04:08:23,0 days 00:00:30
4,2019-09-06 04:09:03,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,0.11,2019-09-06 04:08:23,0 days 00:00:40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4702576,2019-09-03 15:13:29,81,367,8,0.0,0.0,11.7,6.80,6.3,0.0,1078.0,19982.0,1.44,5.0,233.0,236.0,0.08,2019-09-02 19:26:09,0 days 19:47:20
4702577,2019-09-03 15:13:39,81,367,8,0.0,0.0,11.7,6.80,6.3,0.0,1078.0,19982.0,1.44,5.0,233.0,236.0,0.08,2019-09-02 19:26:09,0 days 19:47:30
4702578,2019-09-03 15:13:49,81,367,8,0.0,0.0,11.7,6.80,6.3,0.0,1078.0,19982.0,1.44,5.0,233.0,236.0,0.08,2019-09-02 19:26:09,0 days 19:47:40
4702579,2019-09-03 15:13:59,81,367,8,0.0,0.0,11.7,6.80,6.3,0.0,1078.0,19982.0,1.44,5.0,233.0,236.0,0.08,2019-09-02 19:26:09,0 days 19:47:50


In [278]:
#I convert timeSince to timeSinceSec, the result is in seconds

df['timeSinceSec'] = df['timeSince'].dt.days * 24 * 3600 + df['timeSince'].dt.seconds

In [279]:
# if there are two consecutive 0 it means it is not measuring the seconds, only the minutes

In [280]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4702581 entries, 0 to 4702580
Data columns (total 20 columns):
 #   Column            Dtype          
---  ------            -----          
 0   timestamp         datetime64[ns] 
 1   campaign          int64          
 2   batch             int64          
 3   code              int64          
 4   tbl_speed         float64        
 5   fom               float64        
 6   main_comp         float64        
 7   tbl_fill          float64        
 8   SREL              float64        
 9   pre_comp          float64        
 10  produced          float64        
 11  waste             float64        
 12  cyl_main          float64        
 13  cyl_pre           float64        
 14  stiffness         float64        
 15  ejection          float64        
 16  impurities_total  float64        
 17  start_time        datetime64[ns] 
 18  timeSince         timedelta64[ns]
 19  timeSinceSec      int64          
dtypes: datetime64[ns](2), fl

In [281]:
#Once again I check for missing values (not sure why, just for my peace of mind)

colsMissingVal = df.columns[df.isna().any()].tolist()
df[colsMissingVal].isna().sum()

Series([], dtype: float64)

In [284]:
df

,timestamp,campaign,batch,code,tbl_speed,fom,main_comp,tbl_fill,SREL,pre_comp,produced,waste,cyl_main,cyl_pre,stiffness,ejection,impurities_total,start_time,timeSince,timeSinceSec
0,2019-09-06 04:08:23,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,0.11,2019-09-06 04:08:23,0 days 00:00:00,0
1,2019-09-06 04:08:33,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,0.11,2019-09-06 04:08:23,0 days 00:00:10,10
2,2019-09-06 04:08:43,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,0.11,2019-09-06 04:08:23,0 days 00:00:20,20
3,2019-09-06 04:08:53,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,0.11,2019-09-06 04:08:23,0 days 00:00:30,30
4,2019-09-06 04:09:03,80,368,6,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,0.11,2019-09-06 04:08:23,0 days 00:00:40,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4702576,2019-09-03 15:13:29,81,367,8,0.0,0.0,11.7,6.80,6.3,0.0,1078.0,19982.0,1.44,5.0,233.0,236.0,0.08,2019-09-02 19:26:09,0 days 19:47:20,71240
4702577,2019-09-03 15:13:39,81,367,8,0.0,0.0,11.7,6.80,6.3,0.0,1078.0,19982.0,1.44,5.0,233.0,236.0,0.08,2019-09-02 19:26:09,0 days 19:47:30,71250
4702578,2019-09-03 15:13:49,81,367,8,0.0,0.0,11.7,6.80,6.3,0.0,1078.0,19982.0,1.44,5.0,233.0,236.0,0.08,2019-09-02 19:26:09,0 days 19:47:40,71260
4702579,2019-09-03 15:13:59,81,367,8,0.0,0.0,11.7,6.80,6.3,0.0,1078.0,19982.0,1.44,5.0,233.0,236.0,0.08,2019-09-02 19:26:09,0 days 19:47:50,71270


In [291]:
#Remove categorical data and results from the dataframe

df2 = df.drop(['timestamp', 'campaign', 'batch', 'code', 'start_time', 'timeSince', 'impurities_total'], axis = 1)

In [292]:
df2

,tbl_speed,fom,main_comp,tbl_fill,SREL,pre_comp,produced,waste,cyl_main,cyl_pre,stiffness,ejection,timeSinceSec
0,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,0
1,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,10
2,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,20
3,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,30
4,0.0,0.0,0.0,4.22,0.0,0.0,0.0,0.0,1.37,5.0,2.0,473.0,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4702576,0.0,0.0,11.7,6.80,6.3,0.0,1078.0,19982.0,1.44,5.0,233.0,236.0,71240
4702577,0.0,0.0,11.7,6.80,6.3,0.0,1078.0,19982.0,1.44,5.0,233.0,236.0,71250
4702578,0.0,0.0,11.7,6.80,6.3,0.0,1078.0,19982.0,1.44,5.0,233.0,236.0,71260
4702579,0.0,0.0,11.7,6.80,6.3,0.0,1078.0,19982.0,1.44,5.0,233.0,236.0,71270


In [283]:
#----------------MODEL-----------------------

In [293]:
# Split the data

from sklearn.model_selection import train_test_split


y = df['impurities_total']

X = df2

X.shape, y.shape

((4702581, 13), (4702581,))

In [294]:
#Split the data

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [295]:
# This one took like 40 minutes

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest = RandomForestRegressor(n_estimators=100, random_state=0)

forest.fit(X_train, y_train)

y_pred=forest.predict(X_test)

mean_absolute_error(y_test, y_pred)


0.00024974968876448426

In [296]:
# I reduced the n_estimators and it took a couple of seconds

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest = RandomForestRegressor(n_estimators=1, random_state=0)

forest.fit(X_train, y_train)

y_pred=forest.predict(X_test)

mean_absolute_error(y_test, y_pred)


0.0002803427388266805

In [ ]:
#This results seem too good to be true. I would like to see the results per batch or line by line